In [1]:
import numpy as np
import pandas as pd
# Load EDA
import pandas as pd
import numpy as np
# Load Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as px
plt.style.use('fivethirtyeight')
%matplotlib inline

In [2]:
df1=pd.read_csv("https://raw.githubusercontent.com/AbhinayHinge/Medicine-recommendation/main/Train_Data/drugsComTrain_raw.csv")
df2=pd.read_csv("https://raw.githubusercontent.com/AbhinayHinge/Medicine-recommendation/main/Test_Data/drugsComTest_raw.csv")
df=pd.concat([df1,df2])
df.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [3]:
df.shape

(215063, 7)

In [4]:
# Lets Calculate an Effective Rating

min_rating = df['rating'].min()
max_rating = df['rating'].max()

def scale_rating(rating):
    rating -= min_rating
    rating = rating/(max_rating -1)
    rating *= 5
    rating = int(round(rating,0))
    
    if(int(rating) == 0 or int(rating)==1 or int(rating)==2):
        return 0
    else:
        return 1
    
df['eff_score'] = df['rating'].apply(scale_rating)

In [5]:
# Calculate usefulness score
df['usefulness'] = df['rating'] * df['usefulCount'] * df['eff_score']

# Calculate mean usefulness score
mean_usefulness = df['usefulness'].mean()

# Filter drugs with usefulness score greater than mean
top_drugs = df.loc[df['usefulness'] > mean_usefulness, ['drugName', 'condition', 'usefulness']]

# Sort drugs by usefulness score in descending order and get top 10
top_10_drugs = top_drugs.sort_values(by='usefulness', ascending=False).head(10)

# Print top 10 drugs
print(top_10_drugs)

              drugName      condition  usefulness
33552           Zoloft     Depression       12910
6716        Sertraline     Depression       12910
21708   Levonorgestrel  Birth Control       12470
4249            Mirena  Birth Control       12470
3829        Sertraline     Depression        8541
146145          Zoloft     Depression        8541
58608      Phentermine    Weight Loss        7960
16889         Adipex-P    Weight Loss        7960
5218          Implanon  Birth Control        7300
28789     Etonogestrel  Birth Control        7300


In [6]:
# lets calculate the Number of Useless and Useful Drugs for Each Condition
from ipywidgets import interact
@interact
def check(condition = list(df['condition'].value_counts().index)):
    return df[df['condition'] == condition]['eff_score'].value_counts()

interactive(children=(Dropdown(description='condition', options=('Birth Control', 'Depression', 'Pain', 'Anxie…

In [7]:
# lets check the Most Common Conditions

print("Number of Unique Conditions :", df['condition'].nunique())
df['condition'].value_counts().head(10)

Number of Unique Conditions : 916


Birth Control      38436
Depression         12164
Pain                8245
Anxiety             7812
Acne                7435
Bipolar Disorde     5604
Insomnia            4904
Weight Loss         4857
Obesity             4757
ADHD                4509
Name: condition, dtype: int64

In [8]:
# lets check Drugs, which were useful to Highest Number of Poeple
df[['drugName','usefulCount']][df['usefulCount'] >
                    df['usefulCount'].mean()].sort_values(by = 'usefulCount',
                                        ascending = False).head(10).reset_index(drop = True)

,drugName,usefulCount
0,Zoloft,1291
1,Sertraline,1291
2,Mirena,1247
3,Levonorgestrel,1247
4,Sertraline,949
5,Zoloft,949
6,Adipex-P,796
7,Phentermine,796
8,Citalopram,771
9,Celexa,771


In [9]:
# lets remove all the Duplicates from the Dataset
df = df.drop_duplicates()

In [10]:
@interact
def high_low_rate(condition = list(df['condition'].value_counts().index)):
    top_drugs = df[df['condition'] == condition][['drugName','usefulness']].sort_values(by = 'usefulness',
                                                 ascending = False).reset_index(drop = True)
    top_drugs = top_drugs.drop_duplicates(subset=['drugName'], keep='first').head(5)
    
    
    print("\n Top 5 Drugs")
    print(top_drugs)


 Top 5 Drugs
     drugName  usefulness
0   Buspirone        5265
1      BuSpar        5265
2       Xanax        5260
3  Alprazolam        5260
6   Neurontin        5000
